# bref team batting

In [ ]:
test_url = "https://www.baseball-reference.com/teams/ARI/2024-batting.shtml#players_standard_batting"

In [ ]:
import requests
from bs4 import BeautifulSoup
import polars as pl

In [ ]:
data = requests.get(test_url).text
soup = BeautifulSoup(data, "html.parser")
tables = soup.find_all("table")


In [ ]:
# Assuming we are interested in the first table
table = tables[0]

# Extract headers
headers = [th.text for th in table.find("thead").find_all("th")]
headers.remove("Rk")
headers[-2] = "AllPos"
print(headers)
# Extract rows
rows = []
for row in table.find("tbody").find_all("tr"):
    if "class" not in row.attrs or row.attrs["class"] != ["thead"]:
        cells = row.find_all("td")
        cells = [cell.text.strip() for cell in cells]
        rows.append(cells)
print(rows)
# Create the dataframe using Polars
df = pl.DataFrame(rows, schema=headers, orient="row")
df = df.rename({"WAR": "bWAR"})
df = df.filter(pl.col("OPS+") != "")
df = df.with_columns(
    [
        pl.col("Age").cast(pl.Int32),
        pl.col("G").cast(pl.Int32),
        pl.col("bWAR").cast(pl.Float32),
        pl.col("PA").cast(pl.Int32),
        pl.col("AB").cast(pl.Int32),
        pl.col("R").cast(pl.Int32),
        pl.col("H").cast(pl.Int32),
        pl.col("2B").cast(pl.Int32),
        pl.col("3B").cast(pl.Int32),
        pl.col("HR").cast(pl.Int32),
        pl.col("RBI").cast(pl.Int32),
        pl.col("SB").cast(pl.Int32),
        pl.col("CS").cast(pl.Int32),
        pl.col("BB").cast(pl.Int32),
        pl.col("SO").cast(pl.Int32),
        pl.col("BA").cast(pl.Float32),
        pl.col("OBP").cast(pl.Float32),
        pl.col("SLG").cast(pl.Float32),
        pl.col("OPS").cast(pl.Float32),
        pl.col("OPS+").cast(pl.Int32),
        pl.col("rOBA").cast(pl.Float32),
        pl.col("Rbat+").cast(pl.Int32),
        pl.col("TB").cast(pl.Float32),
        pl.col("GIDP").cast(pl.Int32),
        pl.col("HBP").cast(pl.Int32),
        pl.col("SH").cast(pl.Int32),
        pl.col("SF").cast(pl.Int32),
        pl.col("IBB").cast(pl.Int32),
    ]
)
df
